In [37]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.snowball import SpanishStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import spacy
nlp = spacy.load("es_core_news_md")

In [38]:
data = pd.read_csv("datasets/reviews_filmaffinity_ok.csv")
data

,film_name,gender,film_avg_rate,review_rate,review_title,review_text
0,Ocho apellidos vascos,Comedia,60,3.0,OCHO APELLIDOS VASCOS...Y NINGÚN NOMBRE PROPIO,La mayor virtud de esta película es su existen...
1,Ocho apellidos vascos,Comedia,60,2.0,El perro verde,No soy un experto cinéfilo pero pocas veces me...
2,Ocho apellidos vascos,Comedia,60,2.0,Si no eres de comer mierda... no te comas esta...,Si no eres un incondicional del humor estilo T...
3,Ocho apellidos vascos,Comedia,60,2.0,Aida: The movie,No sé qué está pasando si la gente se deja lle...
4,Ocho apellidos vascos,Comedia,60,2.0,UN HOMBRE SOLO (Julio Iglesias 1987),Pero cuando amanecey me quedo solosiento en el...
...,...,...,...,...,...,...
8598,La isla mínima,Cine negro,72,10.0,magnífica,Buena no lo siguiente. Por fin un film serio c...
8599,La isla mínima,Cine negro,72,3.0,Mucho bombo por ser española pero es igual que...,Me esperaba mucho pero que mucho más.Guión muy...
8600,La isla mínima,Cine negro,72,4.0,¿Quién me quita ahora este mal sabor de boca?,De mal cuerpo como sensación al finalizar de h...
8601,La isla mínima,Cine negro,72,1.0,La isla española,Los que han añadido comentarios os lo han dich...


In [39]:
data['sentiment'] = data['review_rate'].apply(lambda x: 1 if x > 6 else 0)
data

,film_name,gender,film_avg_rate,review_rate,review_title,review_text,sentiment
0,Ocho apellidos vascos,Comedia,60,3.0,OCHO APELLIDOS VASCOS...Y NINGÚN NOMBRE PROPIO,La mayor virtud de esta película es su existen...,0
1,Ocho apellidos vascos,Comedia,60,2.0,El perro verde,No soy un experto cinéfilo pero pocas veces me...,0
2,Ocho apellidos vascos,Comedia,60,2.0,Si no eres de comer mierda... no te comas esta...,Si no eres un incondicional del humor estilo T...,0
3,Ocho apellidos vascos,Comedia,60,2.0,Aida: The movie,No sé qué está pasando si la gente se deja lle...,0
4,Ocho apellidos vascos,Comedia,60,2.0,UN HOMBRE SOLO (Julio Iglesias 1987),Pero cuando amanecey me quedo solosiento en el...,0
...,...,...,...,...,...,...,...
8598,La isla mínima,Cine negro,72,10.0,magnífica,Buena no lo siguiente. Por fin un film serio c...,1
8599,La isla mínima,Cine negro,72,3.0,Mucho bombo por ser española pero es igual que...,Me esperaba mucho pero que mucho más.Guión muy...,0
8600,La isla mínima,Cine negro,72,4.0,¿Quién me quita ahora este mal sabor de boca?,De mal cuerpo como sensación al finalizar de h...,0
8601,La isla mínima,Cine negro,72,1.0,La isla española,Los que han añadido comentarios os lo han dich...,0


In [40]:
data['sentiment'].value_counts()

sentiment
0    4683
1    3920
Name: count, dtype: int64

In [41]:
steamer = SpanishStemmer(ignore_stopwords=True)

def steeming(text):
    stemed_text = re.sub('[^a-zA-Z]', ' ', text)
    stemed_text = stemed_text.lower().split()
    stemed_text = [ steamer.stem(word) for word in stemed_text ]
    stemed_text = ' '.join(stemed_text)

    return stemed_text

def lemmatization(text):
    doc = nlp(text)
    lemmatized_text = ' '.join([token.lemma_ for token in doc if not token.is_stop and not token.is_punct])
    return lemmatized_text

data['stemed_text'] = data['review_text'].apply(steeming)

In [42]:
data

,film_name,gender,film_avg_rate,review_rate,review_title,review_text,sentiment,stemed_text
0,Ocho apellidos vascos,Comedia,60,3.0,OCHO APELLIDOS VASCOS...Y NINGÚN NOMBRE PROPIO,La mayor virtud de esta película es su existen...,0,la mayor virtud de esta pel cul es su existent...
1,Ocho apellidos vascos,Comedia,60,2.0,El perro verde,No soy un experto cinéfilo pero pocas veces me...,0,no soy un expert cin fil pero poc vec me hab a...
2,Ocho apellidos vascos,Comedia,60,2.0,Si no eres de comer mierda... no te comas esta...,Si no eres un incondicional del humor estilo T...,0,si no eres un incondicional del humor estil te...
3,Ocho apellidos vascos,Comedia,60,2.0,Aida: The movie,No sé qué está pasando si la gente se deja lle...,0,no s qu est pas si la gent se dej llev por las...
4,Ocho apellidos vascos,Comedia,60,2.0,UN HOMBRE SOLO (Julio Iglesias 1987),Pero cuando amanecey me quedo solosiento en el...,0,pero cuando amanecey me qued solosient en el f...
...,...,...,...,...,...,...,...,...
8598,La isla mínima,Cine negro,72,10.0,magnífica,Buena no lo siguiente. Por fin un film serio c...,1,buen no lo siguient por fin un film seri con u...
8599,La isla mínima,Cine negro,72,3.0,Mucho bombo por ser española pero es igual que...,Me esperaba mucho pero que mucho más.Guión muy...,0,me esper mucho pero que mucho m s gui n muy vi...
8600,La isla mínima,Cine negro,72,4.0,¿Quién me quita ahora este mal sabor de boca?,De mal cuerpo como sensación al finalizar de h...,0,de mal cuerp como sensaci n al finaliz de hist...
8601,La isla mínima,Cine negro,72,1.0,La isla española,Los que han añadido comentarios os lo han dich...,0,los que han a adid comentari os lo han dich cl...


In [43]:
X = data["stemed_text"].values
y = data["sentiment"].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [44]:
vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [32]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


### Steaming

In [45]:

X_train_prediction = model.predict(X_train)
X_test_prediction = model.predict(X_test)

print("Train accurazy: ", accuracy_score(y_train, X_train_prediction))
print("Test accurazy: ", accuracy_score(y_test, X_test_prediction))


Train accurazy:  0.8791049113629759
Test accurazy:  0.8192911098198722


In [34]:
data2 = pd.read_csv("datasets/dataset.csv")
print("Test accurazy: ", accuracy_score(data2["sentimiento"], model.predict(vectorizer.transform(data2["texto"].apply(steeming)))))


Test accurazy:  0.7233333333333334


In [54]:
model.predict(vectorizer.transform([steeming("me gustara cocinar")]))

array([1])